# Introduction

Today, we'll learn how to perform customer segmentation using **Python** while leveraging the **RFM** framework.

## Dataset

[Online Retail Dataset](https://archive.ics.uci.edu/ml/datasets/Online+Retail)

This Online Retail data set contains all the transactions occurring for a UK-based and registered, non-store online retail between `01-Dec-2010` and `09-Dec-2011`.The company mainly sells unique all-occasion gift-ware. Many customers of the company are wholesalers.

Attribute Information:

- `InvoiceNo`: Invoice number. Nominal. A 6-digit integral number uniquely assigned to each transaction. If this code starts with the letter 'c', it indicates a cancellation.
- `StockCode`: Product (item) code. Nominal. A 5-digit integral number uniquely assigned to each distinct product.
- `Description`: Product (item) name. Nominal.
- `Quantity`: The quantities of each product (item) per transaction. Numeric.
- `InvoiceDate`: Invoice date and time. Numeric. The day and time when a transaction was generated.
- `UnitPrice`: Unit price. Numeric. Product price per unit in sterling (£).
- `CustomerID`: Customer number. Nominal. A 5-digit integral number uniquely assigned to each customer.
- `Country`: Country name. Nominal. The name of the country where a customer resides.

## RFM

**RFM** is commonly used to identify customers who are likely to buy more frequently, spend more, and/or are more likely to return to the company.

RFM stands for the three dimensions:
- `Recency` – How recently did the customer purchase?
- `Frequency` – How often do they purchase?
- `Monetary` – How much do they spend?

# Setup

Import all necessary packages. Normally, you would import these packages in the top of the notebook.

|Packages|Purpose|
|:-|:-|
|<ul><li>`pandas`</li><li>`numpy`</li></ul>|Data wrangling and analysis|
|<ul><li>`matplotlib`</li><li>`plotly.express`</li><li>`seaborn`</li></ul>|Data visualisation|

In [159]:
# !pip install --upgrade plotly
# !pip install plotly-express

In [2]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

# visualisation
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns

# to make subplots
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# better matplotlib plots
%matplotlib inline
%config InlineBackend.figure_format="svg"

# Load data

Let's download the dataset from the link provided above and save it in a `data` folder. You can also save it in a different location if you prefer. 

This is where I store my data.

![image](../../images/tree_directory.png)

Once the dataset is available in your local machine, let's load it using the `pd.read_csv` function to load the data since the data is in **CSV** format

```python
data = pd.read_csv("../../data/OnlineRetail.csv", encoding="ISO-8859-1")
```

In [3]:
data = pd.read_csv("../../data/OnlineRetail.csv", encoding="ISO-8859-1")

**Exercise**

Can you use `shape` to check the shape of the data and print out the number of **rows** and **columns** in the dataset?

In [4]:
# [TODO]
print(f"The dataset contains {data.shape[0]} rows and {data.shape[1]} columns.")

The dataset contains 541909 rows and 8 columns.


**Exercise**

Normally, you would like to **LOOK** at the dataset before doing any work. How do you display **5** random rows from the dataset?

In [5]:
# [TODO]
data.sample(5)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
424674,573277,22359,GLASS JAR KINGS CHOICE,1,10/28/2011 13:18,2.95,14606.0,United Kingdom
356981,568062,35964,FOLKART CLIP ON STARS,36,9/23/2011 13:18,0.39,14014.0,United Kingdom
458962,575855,21671,RED SPOT CERAMIC DRAWER KNOB,2,11/11/2011 12:24,1.45,14527.0,United Kingdom
231326,557263,22326,ROUND SNACK BOXES SET OF4 WOODLAND,192,6/19/2011 11:08,2.55,14646.0,Netherlands
20543,538064,21901,"KEY FOB , BACK DOOR",6,12/9/2010 13:47,0.65,15271.0,United Kingdom


**Exercise**

Can you use `.info` to print out the data types of each column?

Which column(s) has missing data?

In [6]:
# [TODO]
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


**Exercise**

Let's use `describe()` to print out the summary statistics of the dataset.

In [7]:
# [TODO]
data.describe()

,Quantity,UnitPrice,CustomerID
count,541909.000000,541909.000000,406829.000000
mean,9.552250,4.611114,15287.690570
std,218.081158,96.759853,1713.600303
min,-80995.000000,-11062.060000,12346.000000
25%,1.000000,1.250000,13953.000000
50%,3.000000,2.080000,15152.000000
75%,10.000000,4.130000,16791.000000
max,80995.000000,38970.000000,18287.000000


In [8]:
# [TODO]
data.describe(include="object")

,InvoiceNo,StockCode,Description,InvoiceDate,Country
count,541909,541909,540455,541909,541909
unique,25900,4070,4223,23260,38
top,573585,85123A,WHITE HANGING HEART T-LIGHT HOLDER,10/31/2011 14:41,United Kingdom
freq,1114,2313,2369,1114,495478


# EDA

**Exercise**

Which country do the customers from the dataset come from?

**Hint**: Use `.groupby` to group the data by `Country` and `CustomerID` and then use `.nunique()` to count the number of customers per country.

In [12]:
# [TODO]
data.groupby(["Country"])["CustomerID"].nunique().reset_index().sort_values(by="CustomerID", ascending=False).head(10)

,Country,CustomerID
36,United Kingdom,3950
14,Germany,95
13,France,87
31,Spain,31
3,Belgium,25
33,Switzerland,21
27,Portugal,19
19,Italy,15
12,Finland,12
1,Austria,11


**Exercise**

Can you calculate the percentage of customers coming from the UK?

In [14]:
# [TODO]
uk_percentage = data[data.Country == "United Kingdom"]["CustomerID"].nunique()/data["CustomerID"].nunique()
print(f"UK customers account for {uk_percentage*100:.2f}% of all customers")

UK customers account for 90.35% of all customers


Let's visualise the number of customers per country using a **bar plot**.

In [15]:
df = data.groupby(["Country"])["CustomerID"].nunique().reset_index().sort_values(by=["CustomerID"], ascending=False)
fig = px.bar(
    df, x="Country", y="CustomerID", 
    height=600, width=600,
    template="plotly_dark",
)
fig.show()

**Exercise**

What's the percentage of cancelled transactions out of total transactions?

**Hint**: Cancelled transactions are denoted by the `InvoiceNo` column starting with the letter `c`. Convert the `InvoiceNo` column to a `string` and use `.str.contains` (or `str.startswith`) to check if the `InvoiceNo` contains with the letter `c`.

In [19]:
# [TODO]
cancelled_txn_percentage = 1 - (data[~data["InvoiceNo"].str.lower().str.contains("c")].shape[0]/data.shape[0])
# cancelled_txn_percentage = 1 - (data[~data["InvoiceNo"].str.lower().str.startswith("c")].shape[0]/data.shape[0])

print(f"{cancelled_txn_percentage*100:,.2f}% of transactions are cancelled transactions.")

1.71% of transactions are cancelled transactions.


# Data Preprocessing

**Exercise**

What's the data type of the `InvoiceDate` column?

**Hint**: Use `.dtypes` to check the data types of all columns in the dataframe.

In [22]:
# [TODO]
data.dtypes

# data["InvoiceDate"].dtype

InvoiceNo       object
StockCode       object
Description     object
Quantity         int64
InvoiceDate     object
UnitPrice      float64
CustomerID     float64
Country         object
dtype: object

Take 5 samples of the `InvoiceDate` column.

In [169]:
data["InvoiceDate"].sample(5)

277223    7/25/2011 10:28
233818    6/20/2011 15:08
188069    5/12/2011 18:19
443917    11/6/2011 14:29
201777    5/23/2011 15:33
Name: InvoiceDate, dtype: object

**Exercise**

How do we ensure that `InvoiceDate` is correctly of type `date`?

In addition, can you convert `CustomerID` from `float` to `int`. Missing values will be ignored and kept as is. We'll handle them later.

**Hint**: Use `pd.to_datetime` and `astype`

In [24]:
# [TODO]
data = data.assign(
    CustomerID=data["CustomerID"].astype(int, errors="ignore"),
    InvoiceDate=pd.to_datetime(data["InvoiceDate"], format="%m/%d/%Y %H:%M"),
)

**Exercise**

InvoiceNo starting with `c` indicates a cancellation. How do we remove these records from the dataset?

In [23]:
# [TODO]
print(f"Before: {data.shape[0]} rows and {data.shape[1]} columns.")
data = data[~data["InvoiceNo"].str.lower().str.contains("c")]
print(f"After : {data.shape[0]} rows and {data.shape[1]} columns.")

Before: 541909 rows and 8 columns.
After : 532621 rows and 8 columns.


**Exercise**

How do you drop the records with missing values in the dataset?

**NOTE**: There are many ways to deal with missing values in a data science project. Dropping the records with missing values is the simplest method.

In [25]:
# [TODO]
print(f"Before: {data.shape[0]} rows and {data.shape[1]} columns.")
data = data[~data["CustomerID"].isna()]
print(f"After : {data.shape[0]} rows and {data.shape[1]} columns.")

Before: 532621 rows and 8 columns.
After : 397924 rows and 8 columns.


By removing cancelled transactions and missing `CustomerID`, there are no longer any transactions with negative `Quantity`.

In [26]:
data[data["Quantity"] < 0].shape

(0, 8)

**Exercise**

Verify that there are no longer any missing values in the dataset

In [174]:
# [TODO]
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 397924 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    397924 non-null  object        
 1   StockCode    397924 non-null  object        
 2   Description  397924 non-null  object        
 3   Quantity     397924 non-null  int64         
 4   InvoiceDate  397924 non-null  datetime64[ns]
 5   UnitPrice    397924 non-null  float64       
 6   CustomerID   397924 non-null  float64       
 7   Country      397924 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 27.3+ MB


# RFM Analysis

## Recency

**Recency**: How recently did the customer purchase?

In order to answer this question, we need to have an anchor date. We'll take the last `InvoiceDate` as the anchor date.

**Hint**:
```python
LAST_INVOICE_DATE = data["InvoiceDate"].max()
```

In [27]:
LAST_INVOICE_DATE = data["InvoiceDate"].max()
print(f"Anchor date: {LAST_INVOICE_DATE}")

Anchor date: 2011-12-09 12:50:00


**Recency** will be calculated as the number of days between the **anchor date** and the last `InvoiceDate` of each customer.
- Firstly, we need to find the last `InvoiceDate` of each customer.
- Secondly, we'll calculate the time difference between the **anchor date** and the last `InvoiceDate` of each customer. 

In [28]:
# find the last InvoiceDate for each customer
recency_data = data.groupby(["CustomerID"])["InvoiceDate"].agg(["min", "max"])
recency_data.rename(columns={"min": "FirstOrderDate", "max": "LastOrderDate"}, inplace=True)

# calculate the time difference between the anchor and the last InvoiceDate of each customer
recency_data["Recency"] = LAST_INVOICE_DATE - recency_data["LastOrderDate"]
recency_data["Recency"] = recency_data["Recency"].dt.days
recency_data.drop(columns=["FirstOrderDate", "LastOrderDate"], inplace=True)

In [29]:
# have a look at the recency we just calculated
recency_data.head()

,Recency
CustomerID,
12346.0,325
12347.0,1
12348.0,74
12349.0,18
12350.0,309


In [30]:
# Check the shape of recency data
print(f"Recency data contains {recency_data.shape[0]} customers (rows)")

Recency data contains 4339 customers (rows)


Let's take a look at the `recency` distribution. We'll visualise the distribution using both a **histogram** and a **box plot**.

In [31]:
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=("Distribution", "Boxplot"),
    x_title="Number of days between the last transaction and the anchor date",
    y_title="Customer Count",
)

fig.add_histogram(x=recency_data["Recency"], row=1, col=1,)

fig.add_box(x=recency_data["Recency"], row=1, col=2,)

fig.update_layout(height=500, width=1000, showlegend=False, title_text="Recency", template="plotly_dark",)

## Frequency

**Exercise**

**Frequency** will be calculated as the number of times the customer purchased the product.

**Hint**: It will be the count of unique `InvoiceNo` per `CustomerID`.

In [32]:
# [TODO]
frequency_data = data.groupby(["CustomerID"])["InvoiceNo"].agg(["nunique"])
frequency_data.rename(columns={"nunique": "Frequency"}, inplace=True)

In [33]:
# have a look at the frequency we just calculated
frequency_data.head()

,Frequency
CustomerID,
12346.0,1
12347.0,7
12348.0,4
12349.0,1
12350.0,1


In [34]:
# Check the shape of frequency data
print(f"Frequency data contains {frequency_data.shape[0]} customers (rows)")

Frequency data contains 4339 customers (rows)


**Exercise**

You have learned how to visualise **recency** data above. Can you do the same for **frequency**?

In [35]:
# [TODO]

fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=("Distribution", "Boxplot"),
    x_title="Number of purchases",
    y_title="Customer Count",
)

fig.add_histogram(x=frequency_data["Frequency"], row=1, col=1,)

fig.add_box(x=frequency_data["Frequency"], row=1, col=2,)

fig.update_layout(height=500, width=1000, showlegend=False, title_text="Frequency", template="plotly_dark",)

## Monetary

**Exercise**

**Monetary** is will be the total amount of money the customer spent. 

Can you calculate **monetary**?

**Hint**: 
- Use columns `Quantity` and `UnitPrice` to calculate the total amount of money spent by each customer.
- Use `groupby` to group the data by `CustomerID` and then use `sum` to calculate the total amount of money spent by each customer.

In [36]:
# [TODO]

# calculate monetary value of each purchase
data["Amount"] = data["UnitPrice"] * data["Quantity"]

# find the total amount spent for each customer
monetary_data = data.groupby(["CustomerID"])["Amount"].agg(["sum"])

monetary_data.rename(columns={"sum": "Monetary"}, inplace=True)

In [37]:
# Let's take a look at the monetary data we just calculated
monetary_data.head()

,Monetary
CustomerID,
12346.0,77183.60
12347.0,4310.00
12348.0,1797.24
12349.0,1757.55
12350.0,334.40


In [38]:
# Check the shape of monetary data
print(f"Monetary data contains {monetary_data.shape[0]} customers (rows)")

Monetary data contains 4339 customers (rows)


**Exercise**

You have learned how to visualise **recency** data above. Can you do the same for **monetary**?

In [39]:
# [TODO]

fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=("Distribution", "Boxplot"),
    x_title="Total amount spent (£)",
    y_title="Customer Count",
)

fig.add_histogram(x=monetary_data["Monetary"], row=1, col=1,)

fig.add_box(x=monetary_data["Monetary"], row=1, col=2,)


fig.update_layout(height=500, width=1000, showlegend=False, title_text="Monetary", template="plotly_dark")

## Combine RFM

**Exercise**

Can you combine **Recency**, **Frequency** and **Monetary** into 1 single dataframe?

**Hint**: Use `pd.concat`

In [55]:
# [TODO]
rfm_data = pd.concat([recency_data, frequency_data, monetary_data], axis=1)
rfm_data.reset_index(inplace=True)

In [56]:
# Let's take a look at the combined RFM data
rfm_data.head()

,CustomerID,Recency,Frequency,Monetary
0,12346.0,325,1,77183.60
1,12347.0,1,7,4310.00
2,12348.0,74,4,1797.24
3,12349.0,18,1,1757.55
4,12350.0,309,1,334.40


In [57]:
# Check the shape of RFM data
print(f"RFM data contains {rfm_data.shape[0]} customers (rows)")

RFM data contains 4339 customers (rows)


**Recency**, **Frequency** and **Monetary** all appear to contain a lot of outliers. 

Therefore, we'll handle the outliers by capping the values of **Recency**, **Frequency** and **Monetary** at the `10th` and `90th` percentile values respectively.

In [58]:
# treating outliers
FLOORING = 0.1
CAPPING = 0.9

for key in ["Recency", "Frequency", "Monetary"]:
    flooring = rfm_data[key].quantile(FLOORING)
    capping = rfm_data[key].quantile(CAPPING)

    rfm_data[key + "_Original"] = rfm_data[key].copy()
    rfm_data[key] = np.select(
        condlist=[rfm_data[key] <= flooring, rfm_data[key] >= capping],
        choicelist=[flooring, capping],
        default=rfm_data[key]
    )

In [59]:
# Let's take a look at the combined RFM data without outliers
rfm_data.head()

,CustomerID,Recency,Frequency,Monetary,Recency_Original,Frequency_Original,Monetary_Original
0,12346.0,262.0,1.0,3646.164,325,1,77183.60
1,12347.0,4.0,7.0,3646.164,1,7,4310.00
2,12348.0,74.0,4.0,1797.240,74,4,1797.24
3,12349.0,18.0,1.0,1757.550,18,1,1757.55
4,12350.0,262.0,1.0,334.400,309,1,334.40


**Exercise**

Since we keep both the original and fixed RFM data, can you use **box plot** to visualise the changes in the **Recency**, **Frequency** and **Monetary** columns?
The final graph should look like this:

![image](../../images/rfm_original_fixed.png)

In [60]:
# [TODO]
fig = make_subplots(
    rows=3, cols=2,
    subplot_titles=(
        "Original: Recency", "Fixed: Recency", 
        "Original: Frequency", "Fixed: Frequency", 
        "Original: Monetary", "Fixed: Monetary"
    ),
    x_title="Customer Count",
)

for r, key in enumerate(["Recency", "Frequency", "Monetary"]):
    fig.add_box(
        x=rfm_data[f"{key}_Original"], row=(r+1), col=1,
    )

    fig.add_box(
        x=rfm_data[f"{key}"], row=(r+1), col=2,
    )

fig.update_layout(height=700, width=1000, showlegend=False, template="plotly_dark")

fig.show()

Uncomment the codes below to look at another way to visualise the same thing using `seaborn`

In [43]:
# fig, axes = plt.subplots(nrows=3, ncols=2, figsize=(15, 10), sharey=True)

# for r, key in enumerate(["Recency", "Frequency", "Monetary"]):
#     y_axis = axes[r, 0].axes.get_yaxis()
#     y_axis.set_visible(False)

#     axes[r, 0].set_title(f"Original: {key}")
#     sns.boxplot(ax=axes[r, 0], data=rfm_data[f"{key}_Original"], orient="h")

#     axes[r, 1].set_title(f"Fixed: {key}")
#     sns.boxplot(ax=axes[r, 1], data=rfm_data[key], orient="h", color="green")

# RFM Score

RFM scores are defined such that score of `5` and `1` are the `best` and the `worst` respectively.

In [61]:
# most recent purchase should receive the highest score
rfm_data["RScore"] = pd.qcut(rfm_data["Recency"], 5, labels=range(5, 0, -1))

# the more purchases a customer has made, the higher the score
# Some customers will have the same number of purchases, so we'll rank the frequency score based on the first appearance
rfm_data["FScore"] = pd.qcut(rfm_data["Frequency"].rank(method="first"), 5, labels=range(1, 6, 1))

# the more money a customer has spent, the higher the score
rfm_data["MScore"] = pd.qcut(rfm_data["Monetary"], 5, labels=range(1, 6, 1))

In [62]:
# Combine the scores into 1 single label column for each customer
rfm_data["RFMScore"] = (
    rfm_data["RScore"].astype(str) 
    + rfm_data["FScore"].astype(str) 
    + rfm_data["MScore"].astype(str)
)

# Calculate the total RFM Score for each customer
rfm_data["RFMTotal"] = (
    rfm_data["RScore"].astype(int) 
    + rfm_data["FScore"].astype(int) 
    + rfm_data["MScore"].astype(int)
)

In [63]:
# best customers
rfm_data[rfm_data["RFMScore"] == "555"].head()

,CustomerID,Recency,Frequency,Monetary,Recency_Original,Frequency_Original,Monetary_Original,RScore,FScore,MScore,RFMScore,RFMTotal
1,12347.0,4.0,7.0,3646.164,1,7,4310.00,5,5,5,555,15
15,12362.0,4.0,9.0,3646.164,2,10,5226.23,5,5,5,555,15
56,12417.0,4.0,9.0,3646.164,2,9,3649.10,5,5,5,555,15
71,12433.0,4.0,7.0,3646.164,0,7,13375.87,5,5,5,555,15
75,12437.0,4.0,9.0,3646.164,1,18,4951.41,5,5,5,555,15


In [64]:
# worst customers
rfm_data[rfm_data["RFMScore"] == "111"].head()

,CustomerID,Recency,Frequency,Monetary,Recency_Original,Frequency_Original,Monetary_Original,RScore,FScore,MScore,RFMScore,RFMTotal
6,12353.0,203.0,1.0,156.566,203,1,89.00,1,1,1,111,3
14,12361.0,262.0,1.0,189.900,286,1,189.90,1,1,1,111,3
43,12401.0,262.0,1.0,156.566,302,1,84.30,1,1,1,111,3
44,12402.0,262.0,1.0,225.600,322,1,225.60,1,1,1,111,3
77,12441.0,262.0,1.0,173.550,366,1,173.55,1,1,1,111,3


# Segmentation with RFM

Refer to [RFM Segmentation - Business Use](https://documentation.bloomreach.com/engagement/docs/rfm-segmentation) for more information.

|No|Customer Segment|Activity|Actionable Tip|
|:-:|:-|:-|:-|
|1|Champion|Bought recently, order often and spend the most.|Reward them. Can be early adopters of new products. Will promote your brand. Most likely to send referrals.|
|2|Loyal|Orders regularly. Responsive to promotions.|Upsell higher value products. Ask for reviews.|
|3|Potential Loyalist|Recent customers, and spent a good amount.|Offer membership / loyalty program. Keep them engaged. Offer personalised recommendations.|
|4|New Customers|Bought most recently.|Provide on-boarding support, give them early access, start building relationship.|
|5|Promising|Potential loyalist a few months ago. Spends frequently and a good amount. But the last purchase was several weeks ago.|Offer coupons. Bring them back to the platform and keep them engaged. Offer personalised recommendations.|
|6|Core|Standard customers with not too long-ago purchase.|Make limited time offers.|
|7|Needs attention|Core customers whose last purchase happened more than one month ago.|Make limited time offers. Offer personalised recommendations.|
|8|Cannot Lose Them|Made the largest orders, and often. But haven’t returned for a long time.|Win them back via renewals or newer products, don’t lose them to competition. Talk to them if necessary. Spend time on highest possible personalisation.|
|9|At Risk|Similar to “Can’t lose them but losing” but with smaller monetary and frequency value.|Provide helpful resources on the site. Send personalised emails.|
|10|Hibernating|Made their last purchase a long time ago but in the last 4 weeks either visited the site or opened an email.|Make subject lines of emails very personalised. Revive their interest by a specific discount on a specific product.|
|11|Lost|Made last purchase long time ago and didn’t engage at all in the last 4 weeks.|Revive interest with reach out campaign. Ignore otherwise.|

**Exercise**

Use the segmentation map below to perform customer segmentation.

```python
segmentation_map = {
    r'555|554|544|545|454|455|445': 'Champions',
    r'543|444|435|355|354|345|344|335': 'Loyal',
    r'553|551|552|541|542|533|532|531|452|451|442|441|431|453|433|432|423|353|352|351|342|341|333|323': 'Potential Loyalist',
    r'512|511|422|421|412|411|311': 'New Customers',
    r'525|524|523|522|521|515|514|513|425|424|413|414|415|315|314|313': 'Promising',
    r'535|534|443|434|343|334|325|324': 'Need Attention',
    r'331|321|312|221|213|231|241|251': 'About To Sleep',
    r'255|254|245|244|253|252|243|242|235|234|225|224|153|152|145|143|142|135|134|133|125|124': 'At Risk',
    r'155|154|144|214|215|115|114|113': 'Cannot Lose Them',
    r'332|322|233|232|223|222|132|123|122|212|211': 'Hibernating',
    r'111|112|121|131|141|151': 'Lost',
}
```

The segment should be stored in a new column `RFMSegment` in the dataframe.

```python
rfm_data['RFMSegment'] = ...
```

In [66]:
# [TODO]
segmentation_map = {
    r'555|554|544|545|454|455|445': 'Champions',
    r'543|444|435|355|354|345|344|335': 'Loyal',
    r'553|551|552|541|542|533|532|531|452|451|442|441|431|453|433|432|423|353|352|351|342|341|333|323': 'Potential Loyalist',
    r'512|511|422|421|412|411|311': 'New Customers',
    r'525|524|523|522|521|515|514|513|425|424|413|414|415|315|314|313': 'Promising',
    r'535|534|443|434|343|334|325|324': 'Need Attention',
    r'331|321|312|221|213|231|241|251': 'About To Sleep',
    r'255|254|245|244|253|252|243|242|235|234|225|224|153|152|145|143|142|135|134|133|125|124': 'At Risk',
    r'155|154|144|214|215|115|114|113': 'Cannot Lose Them',
    r'332|322|233|232|223|222|132|123|122|212|211': 'Hibernating',
    r'111|112|121|131|141|151': 'Lost',
}
rfm_data['RFMSegment'] = rfm_data["RFMScore"].replace(segmentation_map, regex=True)

**Exercise**

Let's calculate the `median`, `mean` and `std` of **Recency**, **Frequency**, **Monetary** for each segment in `RFMSegment` column.

In [67]:
# [TODO]
rfm_data.groupby(["RFMSegment"])[
    "Recency",
    "Frequency",
    "Monetary"
].agg(["median", "mean", "std"]).reset_index()

RFMSegment Recency                        Frequency            \
                        median        mean        std    median      mean   
0       About To Sleep    70.0   86.530466  43.600190       1.0  1.279570   
1              At Risk   122.0  141.326877  60.257273       3.0  3.644068   
2     Cannot Lose Them   238.0  214.739130  56.974810       1.0  2.054348   
3            Champions     8.0   11.278571   8.462834       8.0  7.401190   
4          Hibernating   128.0  141.489796  72.013071       1.0  1.553936   
5                 Lost   262.0  244.184426  25.965994       1.0  1.065574   
6                Loyal    37.0   36.112150  18.794003       4.5  4.983645   
7       Need Attention    30.0   33.123675  19.038098       3.0  3.098940   
8        New Customers    24.0   27.724265  16.792671       1.0  1.069853   
9   Potential Loyalist    22.0   26.414692  19.188305       2.0  2.521327   
10           Promising    16.0   24.551471  21.114790       1.0  1.323529   

              Monetary                           
         std    median         mean         std  
0   0.629587   210.350   290.584781  183.242283  
1   1.738121  1169.940  1462.443673  919.897688  
2   2.180655  1074.910  1342.882761  859.477292  
3   1.846880  2984.625  2772.753052  919.838516  
4   0.636487   374.850   414.796717  178.656649  
5   0.271514   178.955   221.241754   85.831753  
6   1.798311  1611.025  1881.199570  934.508069  
7   0.901594   942.260  1054.008512  494.519694  
8   0.255369   199.640   238.059029   94.209998  
9   1.162874   553.400   543.289024  213.197928  
10  0.469552   664.460   873.320309  720.996698

# Visualisation

**Exercise**

The distribution of Recency, Frequency, and Monetary need to follow the same direction since we assign 1 and 5 for the worst and the best customer respectively.

Can you verify this by plotting the distribution?

In [68]:
# [TODO]
fig = make_subplots(
    rows=3, cols=2,
    subplot_titles=(
        "Recency: Distribution", "Recency: Boxplot", 
        "Frequency: Distribution", "Frequency: Boxplot", 
        "Monetary: Distribution", "Monetary: Boxplot"
    ),
    x_title="Customer Count",
)

for r, key in enumerate(["Recency", "Frequency", "Monetary"]):
    fig.add_histogram(
        x=rfm_data[f"{key}"], row=(r+1), col=1, nbinsx=5,
    )

    fig.add_box(
        x=rfm_data[f"{key}"], row=(r+1), col=2,
    )

fig.update_layout(height=700, width=1000, showlegend=False, template="plotly_dark",)

fig.show()

Uncomment the codes below to look at another way to visualise the same thing using `seaborn`

In [69]:
# fig, axes = plt.subplots(nrows=3, ncols=2, figsize=(15, 10))

# for r, key in enumerate(["Recency", "Frequency", "Monetary"]):
#     sns.histplot(ax=axes[r, 0], data=rfm_data[key], bins=5, kde=True, legend=False)

#     y_axis_ax1 = axes[r, 1].axes.get_yaxis()
#     y_axis_ax1.set_visible(False)
#     axes[r, 1].set_xlabel(f"{key}")
#     sns.boxplot(ax=axes[r, 1], data=rfm_data[key], orient="h")

Let's see if Recency, Frequency and Monetary are correlated to each other using a **heatmap**.

In [71]:
fig = px.imshow(
    rfm_data[["Recency", "Frequency", "Monetary"]].corr(),
    text_auto=True,
    template="plotly_dark",
)
fig.show()

Let's visualise the customers based on the RFM Segment on a 3D **scatter plot**.

In [72]:
fig = px.scatter_3d(
    data_frame=rfm_data,
    x="Recency",
    y="Frequency",
    z="Monetary",
    color="RFMSegment",
    template="plotly_dark",
    height=700,
    width=700,
)

fig.show()

Let's draw a **treemap** of the customers based on the RFM Segment

In [73]:
df = rfm_data["RFMSegment"].value_counts().reset_index()
df.columns = ["RFMSegment", "Count"]
fig = px.treemap(
    data_frame=df, 
    path=["RFMSegment"], 
    values="Count", 
    template="plotly_dark",
    width=800, 
    height=400,
)
fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))
fig.show()

**Exercise**

**VERY HARD** 🤯

Plot the distribution of **Monetary** for each combination of **Recency** and **Frequency** score. 

**NOTE**: Ideally, we are expecting to see a diagonal line having the green bar increasing from 1 to 5 for **Monetary**.

**Hint**: Use loops to loop through different combination of values of R, F.

The final result should look like this:

![image](../../images/rfm_M_for_each_R_and_F.png)

In [208]:
# [TODO]
fig = make_subplots(
    rows=5, cols=5,
    x_title="Recency",
    y_title="Frequency",
    shared_yaxes=True,
)

for r in range(1, 6):
    for f in range(1, 6):
        # prepare data
        df = rfm_data.query(f"RScore == {r} & FScore == {f}").groupby(["MScore"])["CustomerID"].count().reset_index().sort_values(by="CustomerID", ascending=False)
        df["color"] = np.where(df["CustomerID"] == df["CustomerID"].max(), "green", "grey")
        
        # visualise
        fig.add_bar(
            x=df["MScore"],
            y=df["CustomerID"],
            text=df["CustomerID"],
            marker_color=df["color"],
            row=r, col=f,
        )

fig.update_layout(
    height=1200, 
    width=1200, 
    showlegend=False, 
    template="plotly_dark", 
    title_text="M distribution for each F and R",
)

fig.show()

Uncomment the codes below to look at another way to visualise the same thing using `seaborn`

In [74]:
# fig, axes = plt.subplots(nrows=5, ncols=5,
#                          sharex=False, sharey=True,
#                          figsize=(10, 10))

# for r in range(1, 6):
#     for f in range(1, 6):
#         y = rfm_data[(rfm_data["RScore"] == r) & (rfm_data["FScore"] == f)]["MScore"].value_counts().sort_index()
#         x = y.index
#         ax = axes[r - 1, f - 1]
#         bars = ax.bar(x, y, color="grey")
#         if r == 5:
#             if f == 3:
#                 ax.set_xlabel(f"{f}\nF", va='top')
#             else:
#                 ax.set_xlabel(f"{f}\n", va='top')
#         if f == 1:
#             if r == 3:
#                 ax.set_ylabel(f"R\n{r}")
#             else:
#                 ax.set_ylabel(r)
#         ax.set_frame_on(False)
#         ax.tick_params(left=False, labelleft=False, bottom=False)
#         ax.set_xticks(x)
#         ax.set_xticklabels(x, fontsize=8)

#         for bar in bars:
#             value = bar.get_height()
#             if value == y.max():
#                 bar.set_color("green")
#             ax.text(bar.get_x() + bar.get_width() / 2,
#                     value,
#                     int(value),
#                     ha="center",
#                     va="bottom",
#                     color="k")
# fig.suptitle("M distribution for each F and R", fontsize=14)
# plt.tight_layout()
# plt.show()